### Replace audio

![](pictures/DYx0E.png)

```
ffmpeg -i video.mp4 -i audio.wav -map 0:v -map 1:a -c:v copy -shortest output.mp4
```

* Опция `-map` позволяет вам вручную выбирать потоки / треки. Смотрите [FFmpeg Wiki: Map](https://trac.ffmpeg.org/wiki/Map) для получения дополнительной информации.
* В этом примере используется `-c:v copy` для потокового копирования (мультипликации) видео. Перекодирование видео не происходит. Качество сохраняется, а процесс происходит быстро.
    * Если ваш входной аудиоформат совместим с выходным форматом, измените `-c:v copy` на `-c copy` для потокового копирования как видео, так и аудио.
    * Если вы хотите перекодировать видео и аудио, то удалите `-c:v copy` / `-c copy`.
* Опция `-shortest` сделает вывод такой же продолжительности, как и самый короткий ввод.


### Add audio

![](pictures/eyCVy.png)

```
ffmpeg -i video.mkv -i audio.mp3 -map 0 -map 1:a -c:v copy -shortest output.mkv
```

### Mixing/combining two audio inputs into one

![](pictures/2P6Ls.png)


Используйте видео из video.mkv. Микшируйте аудио из video.mkv и audio.m4a с помощью фильтра [amerge filter](http://ffmpeg.org/ffmpeg-filters.html#amerge):

```
ffmpeg -i video.mkv -i audio.m4a -filter_complex "[0:a][1:a]amerge=inputs=2[a]" -map 0:v -map "[a]" -c:v copy -ac 2 -shortest output.mkv
```

Больше информации [FFmpeg Wiki: Audio Channels](https://trac.ffmpeg.org/wiki/AudioChannelManipulation)

### Generate silent audio

```
ffmpeg -i video.mp4 -f lavfi -i anullsrc=channel_layout=stereo:sample_rate=44100 \
-c:v copy -shortest output.mp4
```

Вы можете использовать [фильтр anullsrc](https://ffmpeg.org/ffmpeg-filters.html#anullsrc) для создания бесшумного аудиопотока. Фильтр позволяет вам выбрать желаемую компоновку канала (моно, стерео, 5.1 и т.д.) и частоту дискретизации.

### Для объединения двух аудио файлов с помощью ffmpeg, можно использовать команду "concat".

Сначала нужно создать текстовый файл, в котором будут перечислены пути к файлам, которые нужно объединить. Например, создайте файл "list.txt" и добавьте в него следующее содержимое:
```
file '/path/to/file1.mp3'
file '/path/to/file2.mp3'
```
Затем запустите команду ffmpeg, указав путь к созданному файлу "list.txt":
Copy code
```powershell
ffmpeg -f concat -safe 0 -i list.txt -c copy output.mp3
```
Где:

* "-f concat" указывает, что мы используем функцию объединения файлов;
* "-safe 0" отключает проверку безопасности, чтобы ffmpeg мог использовать файлы, находящиеся вне текущего каталога;
* "-i list.txt" указывает путь к файлу, содержащему список файлов для объединения;
* "-c copy" указывает ffmpeg скопировать аудио потоки без изменений;
* "output.mp3" - это имя выходного файла.

После выполнения этой команды, вы получите один файл "output.mp3", который будет содержать объединенные аудио файлы.

---
---
---

In [1]:
import os
from pathlib import Path
import subprocess
from textwrap import shorten

In [4]:
path_to_folder = Path('a:/')
file_list = list(path_to_folder.iterdir())
for index, item in enumerate(file_list):
    print(f'{index:02} {item.name}')

video = file_list[3]
audio = file_list[2]
print('-'*80)
print(f'\nvideo file:\n\t{video.name}')
print(f'\naudio file:\n\t{audio.name}')

name = video.name.replace(video.suffix, '')
symbols = ' abcdefghijklmnopqrstuvwxyzабвгдеёжзийклмнопрстуфхцчшщъыьэюя0123456789'
name = ''.join(filter(lambda x: x if x.lower() in symbols else '', name))

out_name = shorten(name, 40, placeholder='').replace(' ', '_') + '_out' + video.suffix
out_file = path_to_folder.joinpath(out_name)
print('-'*80)
print(f'\nout file: \n\t{out_file}')

00 $RECYCLE.BIN
01 ABUS Я разочарован [mXLBzDmnpyc].webm
02 fd9a8a8939a049478a9faeb84f5e8f2f.mp3
03 VFX Artists React to Avatar 2 CGi (ft. Weta FX) [c4Gd0bR2kb4].webm
04 Двойной рассказ о Splinter Cell： Double Agent [Разбор Полётов] [FB5Dhj9BleM].webm
05 Этому миру есть альтернативы ⧸⧸ Комолов & Бузгалин. Диалог [VN4MEVIctbs].webm
--------------------------------------------------------------------------------

video file:
	VFX Artists React to Avatar 2 CGi (ft. Weta FX) [c4Gd0bR2kb4].webm

audio file:
	fd9a8a8939a049478a9faeb84f5e8f2f.mp3
--------------------------------------------------------------------------------

out file: 
	a:\VFX_Artists_React_to_Avatar_2_CGi_ft_out.webm


In [5]:
# С помощью ffmpeg заменяем/добавляем аудио дорожку

args = [
    'ffmpeg', '-y',
    '-i', video,
    '-i', audio,
    '-map', '0',
    '-map', '1:a',
    '-c:v', 'copy',
    '-c:a:0', 'copy',
    # '-c:a:1', 'aac',
    # '-b:a:1', '320k',
    '-shortest',
    out_file
]
subprocess.run(args, shell=True, check=True)
# print(' '.join(map(lambda x: f'"{str(x)}"' if type(x) is not str else x, args)))

CompletedProcess(args=['ffmpeg', '-y', '-i', WindowsPath('a:/VFX Artists React to Avatar 2 CGi (ft. Weta FX) [c4Gd0bR2kb4].webm'), '-i', WindowsPath('a:/fd9a8a8939a049478a9faeb84f5e8f2f.mp3'), '-map', '0', '-map', '1:a', '-c:v', 'copy', '-c:a:0', 'copy', '-shortest', WindowsPath('a:/VFX_Artists_React_to_Avatar_2_CGi_ft_out.webm')], returncode=0)

---
---